In [5]:
install.packages('TSclust', repos="http://cran.rstudio.com/")

Installing package into 'C:/Users/Tom Work/Documents/R/win-library/3.3'
(as 'lib' is unspecified)
also installing the dependencies 'splus2R', 'ifultools', 'proxy', 'clv', 'rgl', 'misc3d', 'wmtsa', 'pdc', 'locpol', 'dtw', 'longitudinalData'



package 'splus2R' successfully unpacked and MD5 sums checked
package 'ifultools' successfully unpacked and MD5 sums checked
package 'proxy' successfully unpacked and MD5 sums checked
package 'clv' successfully unpacked and MD5 sums checked
package 'rgl' successfully unpacked and MD5 sums checked
package 'misc3d' successfully unpacked and MD5 sums checked
package 'wmtsa' successfully unpacked and MD5 sums checked
package 'pdc' successfully unpacked and MD5 sums checked
package 'locpol' successfully unpacked and MD5 sums checked
package 'dtw' successfully unpacked and MD5 sums checked
package 'longitudinalData' successfully unpacked and MD5 sums checked
package 'TSclust' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\Tom Work\AppData\Local\Temp\RtmpwtVNrf\downloaded_packages


In [6]:
library('TSclust')

Loading required package: wmtsa
Loading required package: pdc
Loading required package: cluster
